In [192]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import re
import time

Initial page exploration

In [207]:
url = 'https://www.opensecrets.org/races/candidates?cycle=2020&id=TN07&spec=N'

response = requests.get(url)

In [208]:
response.status_code

200

In [209]:
soup = BeautifulSoup(response.text)

In [210]:
soup.findAll('div', attrs = {'Members--list-item'})

# incumbant has 'a' tag, others have 'h2'
# peel off incumbant yes/no winner and affiliation with regex (flatten list by choosing index 0) 
# and then index[1] to get letter


[<div class="Members--list-item">
 <div class="Members--bio u-richtext">
 <h2><strong>
 <a href="/members-of-congress/mark-green/summary?cid=N00041873">Mark Green (R) • <i>Incumbent</i></a>
  • <span class="winner"><i>Winner</i></span> <br/>
 <span class="Members--vote-pct">(69.9% of vote)</span>
 </strong></h2>
 </div>
 <table border="0" cellpadding="1" cellspacing="0" class="Members--table">
 <tr>
 <td>Raised:</td>
 <td class="Members--number">$1,194,960</td>
 <td width="33%">
 <div class="Members--bar-wrapper">
 <div class="Members--bar Members--bar-indivs" style="width:100.0%;"></div>
 </div>
 </td>
 </tr>
 <tr>
 <td>Spent:</td>
 <td class="Members--number">$935,487</td>
 <td width="33%">
 <div class="Members--bar-wrapper">
 <div class="Members--bar Members--bar-indivs" style="width:78.28599300862228%;"></div>
 </div>
 </td>
 </tr>
 <tr>
 <td>Cash on Hand:</td>
 <td class="Members--number">$287,889</td>
 <td width="33%">
 <div class="Members--bar-wrapper">
 <div class="Members--bar

In [211]:
s = soup.findAll('div', attrs = {'Members--list-item'})

In [212]:
s

[<div class="Members--list-item">
 <div class="Members--bio u-richtext">
 <h2><strong>
 <a href="/members-of-congress/mark-green/summary?cid=N00041873">Mark Green (R) • <i>Incumbent</i></a>
  • <span class="winner"><i>Winner</i></span> <br/>
 <span class="Members--vote-pct">(69.9% of vote)</span>
 </strong></h2>
 </div>
 <table border="0" cellpadding="1" cellspacing="0" class="Members--table">
 <tr>
 <td>Raised:</td>
 <td class="Members--number">$1,194,960</td>
 <td width="33%">
 <div class="Members--bar-wrapper">
 <div class="Members--bar Members--bar-indivs" style="width:100.0%;"></div>
 </div>
 </td>
 </tr>
 <tr>
 <td>Spent:</td>
 <td class="Members--number">$935,487</td>
 <td width="33%">
 <div class="Members--bar-wrapper">
 <div class="Members--bar Members--bar-indivs" style="width:78.28599300862228%;"></div>
 </div>
 </td>
 </tr>
 <tr>
 <td>Cash on Hand:</td>
 <td class="Members--number">$287,889</td>
 <td width="33%">
 <div class="Members--bar-wrapper">
 <div class="Members--bar

In [213]:
# state, only do this once for each page - 
soup.find('a', attrs = {'class': 'SubNav-link--active'}).text.split()[0:-2]

"['Tennessee']"

In [204]:
# district, only do this once for each page
soup.find('a', attrs = {'class': 'SubNav-link--active'}).text.split()[-1]

'07'

In [ ]:
# for first candidate on page (only one with 'a' tag), all information
s[0].find('a').text.split(' (')[0] # candidate name
re.findall(r'\([A-Z]\)', s[0].find('a').text)[0][1] # party affiliation
s[0].find('a').text.split()[-1] # incumbant always with 'a' tag
s[0].find('span', attrs = {'class': 'winner'}).text # if present then winner
s[0].find('span', attrs = {'class': 'Members--vote-pct'}).text.strip('(').strip(')').replace('% of vote', '') # % of vote
s[0].findAll('td', attrs = {'class': 'Members--number'})[0].text.replace('$', '') # money raised
s[0].findAll('td', attrs = {'class': 'Members--number'})[1].text.replace('$', '') # money spent

In [136]:
# for all other candidates on page, put into loop with range of len(s)
s[1].find('strong').text.split(' (')[0].strip() # candidate name
re.findall(r'\([A-Z]\)', s[0].find('strong').text)[0][1] # party affiliation
s[1].find('span', attrs = {'class': 'winner'}).text # if present then winner
s[1].find('span', attrs = {'class': 'Members--vote-pct'}).text.strip('(').strip(')').replace('% of vote', '') # % of vote
s[1].findAll('td', attrs = {'class': 'Members--number'})[0].text.replace('$', '') # money raised
s[1].findAll('td', attrs = {'class': 'Members--number'})[1].text.replace('$', '') # money spent

'Kiran Sreepada'

In [177]:
s[1].find('strong').text.split(' (')[0].strip() # candidate name

'Kiran Sreepada'

In [185]:
# for single page

candidate_df = pd.DataFrame()
candidate_results = {}

state_name = soup.find('a', attrs = {'class': 'SubNav-link--active'}).text.split()[0:-2]
district = soup.find('a', attrs = {'class': 'SubNav-link--active'}).text.split()[-1]

candidates = soup.findAll('div', attrs = {'Members--list-item'})

for candidate in candidates:
    candidate_results['state_name'] = state_name
    candidate_results['district'] = district
    if candidate.find('a'):
        candidate_results['candidate_name'] = candidate.find('a').text.split(' (')[0]
        candidate_results['party'] = re.findall(r'\([A-Z]\)', candidate.find('a').text)[0][1]
        candidate_results['incumbent'] = candidate.find('a').text.split()[-1]
    else:
        candidate_results['candidate_name'] = candidate.find('strong').text.split(' (')[0].strip()
        candidate_results['party'] = re.findall(r'\([A-Z]\)', s[0].find('strong').text)[0][1]
        candidate_results['incumbent'] = 'not incumbent'
    if candidate.find('span', attrs = {'class': 'winner'}):
        candidate_results['winner'] = 1
    else:
        candidate_results['winner'] = 0
    candidate_results['perc_vote'] = candidate.find('span', attrs = {'class': 'Members--vote-pct'})\
        .text.strip('(').strip(')').replace('% of vote', '')
    candidate_results['money_raised'] = candidate.findAll('td', attrs = {'class': 'Members--number'})[0]\
        .text.replace('$', '')
    candidate_results['money_spent'] = candidate.findAll('td', attrs = {'class': 'Members--number'})[1]\
        .text.replace('$', '')
    can_res = pd.DataFrame([candidate_results])
    candidate_df = pd.concat([candidate_df, can_res])


In [186]:
candidate_df

,state_name,district,candidate_name,party,incumbent,winner,perc_vote,money_raised,money_spent
0,Tennessee,07,Mark Green,R,Incumbent,1,69.9,"1,194,960","935,487"
0,Tennessee,07,Kiran Sreepada,R,not incumbent,0,27.3,"206,644","207,191"
0,Tennessee,07,Ronald Brown,R,not incumbent,0,2.2,"1,750",0
0,Tennessee,07,Scott Vieira Jr,R,not incumbent,0,0.6,655,"1,049"


In [187]:
urls = pd.read_csv('../data/dist_urls.csv')
urls.head(1)

,url
0,https://www.opensecrets.org/races/candidates?c...


In [222]:
# needed more reliable method to get state and district
s_d = re.findall(r'[A-Z]{2}[0-9]{2}', urls['url'][0])[0]
state = s_d[:2]
print(state)
district = s_d[2:]
print(district)

AL
01


In [230]:
candidate_df = pd.DataFrame()
candidate_results = {}

for url in urls['url']:
    response = requests.get(url)
    soup = BeautifulSoup(response.text)

    time.sleep(0.3)
    
    s_d = re.findall(r'[A-Z]{2}[0-9]{2}', url)[0]
    state = s_d[:2]
    district = s_d[2:]

    candidates = soup.findAll('div', attrs = {'Members--list-item'})

    for candidate in candidates:
        candidate_results['state_name'] = state
        candidate_results['district'] = district
        if candidate.find('a'):
            candidate_results['candidate_name'] = candidate.find('a').text.split(' (')[0]
            candidate_results['party'] = re.findall(r'\([A-Z]\)', candidate.find('a').text)[0][1]
            candidate_results['incumbent'] = candidate.find('a').text.split()[-1]
        else:
            candidate_results['candidate_name'] = candidate.find('strong').text.split(' (')[0].strip()
            candidate_results['party'] = re.findall(r'\([A-Z]\)', s[0].find('strong').text)[0][1]
            candidate_results['incumbent'] = 'not incumbent'
        if candidate.find('span', attrs = {'class': 'winner'}):
            candidate_results['winner'] = 1
        else:
            candidate_results['winner'] = 0
        if candidate.find('span', attrs = {'class': 'Members--vote-pct'}):
            candidate_results['perc_vote'] = candidate.find('span', attrs = {'class': 'Members--vote-pct'})\
                .text.strip('(').strip(')').replace('% of vote', '')
        candidate_results['money_raised'] = candidate.findAll('td', attrs = {'class': 'Members--number'})[0]\
            .text.replace('$', '').replace(',' , '')
        candidate_results['money_spent'] = candidate.findAll('td', attrs = {'class': 'Members--number'})[1]\
            .text.replace('$', '').replace(',' , '')
        can_res = pd.DataFrame([candidate_results])
        candidate_df = pd.concat([candidate_df, can_res])
    
    print(f'finished {state} {district}')

finished AL 01
finished AL 02
finished AL 03
finished AL 04
finished AL 05
finished AL 06
finished AL 07
finished AK 01
finished AZ 01
finished AZ 02
finished AZ 03
finished AZ 04
finished AZ 05
finished AZ 06
finished AZ 07
finished AZ 08
finished AZ 09
finished AR 01
finished AR 02
finished AR 03
finished AR 04
finished CA 01
finished CA 02
finished CA 03
finished CA 04
finished CA 05
finished CA 06
finished CA 07
finished CA 08
finished CA 09
finished CA 10
finished CA 11
finished CA 12
finished CA 13
finished CA 14
finished CA 15
finished CA 16
finished CA 17
finished CA 18
finished CA 19
finished CA 20
finished CA 21
finished CA 22
finished CA 23
finished CA 24
finished CA 25
finished CA 26
finished CA 27
finished CA 28
finished CA 29
finished CA 30
finished CA 31
finished CA 32
finished CA 33
finished CA 34
finished CA 35
finished CA 36
finished CA 37
finished CA 38
finished CA 39
finished CA 40
finished CA 41
finished CA 42
finished CA 43
finished CA 44
finished CA 45
finished C

In [231]:
candidate_df.shape

(892, 9)

In [232]:
candidate_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 892 entries, 0 to 0
Data columns (total 9 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   state_name      892 non-null    object
 1   district        892 non-null    object
 2   candidate_name  892 non-null    object
 3   party           892 non-null    object
 4   incumbent       892 non-null    object
 5   winner          892 non-null    int64 
 6   perc_vote       892 non-null    object
 7   money_raised    892 non-null    object
 8   money_spent     892 non-null    object
dtypes: int64(1), object(8)
memory usage: 69.7+ KB


In [233]:
candidate_df[['perc_vote', 'money_raised', 'money_spent']] = candidate_df[['perc_vote', 'money_raised', 'money_spent']].apply(pd.to_numeric)

In [234]:
candidate_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 892 entries, 0 to 0
Data columns (total 9 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   state_name      892 non-null    object 
 1   district        892 non-null    object 
 2   candidate_name  892 non-null    object 
 3   party           892 non-null    object 
 4   incumbent       892 non-null    object 
 5   winner          892 non-null    int64  
 6   perc_vote       892 non-null    float64
 7   money_raised    892 non-null    int64  
 8   money_spent     892 non-null    int64  
dtypes: float64(1), int64(3), object(5)
memory usage: 69.7+ KB


In [235]:
candidate_df.to_csv('../data/candidates.csv', index = False)